# Path Flow Estimator

In [21]:
import gurobipy as gp  
from gurobipy import GRB  
import pickle  
import model_param  # 计算参数的一堆函数

## 数据部分

读取基础数据文件（pNEUMA）

In [4]:
pkl_file = open('data/df_route.pkl', 'rb')  # 这是预先处理好的数据文件 
df_route = pickle.load(pkl_file)  
pkl_file = open('data/edges_used.pkl', 'rb')  
df_link = pickle.load(pkl_file) 

num_route, num_link = len(df_route), len(df_link)
print(f'num_route: {num_route}, num_link: {num_link}') 

num_route: 460, num_link: 402


In [5]:
df_route.head(2)

,OD_id,o_lon_col,o_lat_col,d_lon_col,d_lat_col,o_node_id,d_node_id,route_node,route_grid,route_flow,higher_routes,route_flow_true
0,0,0,14,1,14,97835127,97834824,"[97835127, 97834909, 633562896, 97834824]","[(0, 14), (0, 15), (1, 15), (1, 14)]",2,"[1, 2, 3]",5
1,0,0,14,1,14,97835127,633562897,"[97835127, 97834909, 633562896, 97834824, 9783...","[(0, 14), (0, 15), (1, 15), (1, 14), (1, 14), ...",1,[],1


route_link_indicator

In [8]:
# get the route_link_indicator (可以打开脚本文件查看具体代码)
route_link_indicator = model_param.get_indicator(num_route, num_link, df_route, df_link) 

给出检测器布局方案（本案例我们采用最大化可观测路径下的检测器布局方案）

In [9]:
# number of sensors
num_sensor = 40

# scanned_links & observable_routes (read from pickle files) 
pkl_file = open('data/exp_design/link_scanned_{}_sensors.pkl'.format(num_sensor), 'rb')
scanned_links = pickle.load(pkl_file) 

pkl_file = open('data/exp_design/route_identified_{}_sensors.pkl'.format(num_sensor), 'rb')
observable_routes = pickle.load(pkl_file) 

In [12]:
# 检测器对
sensor_od_pair = [(sensor_1, sensor_2) for sensor_1 in scanned_links for sensor_2 in scanned_links 
                  if sensor_1 != sensor_2]

In [10]:
# route_sensor_od_indicator
route_sensor_od_indicator = model_param.get_route_sensor_od_indicator(num_route, 
                                                                      num_link,  
                                                                      route_link_indicator,  
                                                                      scanned_links,  
                                                                      df_route,  
                                                                      df_link) 

In [11]:
# 6 route flow  
route_flow = list(df_route['route_flow'])  

In [15]:
# sensor od flow 
sensor_od = model_param.get_sensor_OD_flow(num_route, num_link, 
                                           route_sensor_od_indicator, 
                                           scanned_links, df_route)  


In [13]:
# 路段流量
df_link = model_param.get_link_flow(df_link, df_route, route_link_indicator)
link_flow = list(df_link['link_flow'])

In [16]:
# 路段长度
link_length = list(df_link['length'])

In [17]:
# 路段行程时间（自己写的一个阻抗函数）
error_ratio = 0.01 #  
link_tt = [model_param.bpr_link_tt(link_flow[link], link_length[link], error_ratio) 
           for link in range(num_link)] 

In [18]:
# 路径行程时间
route_tt = [] 
for route in range(num_route):
    tmp = sum([link_tt[link] * route_link_indicator[route, link] for link in range(num_link)])
    route_tt.append(tmp) 

In [20]:
# 13 OD对的列表 
od_list = list(df_route.drop_duplicates('OD_id')['OD_id']) 

# 构建每个OD对与路径编号的关系 
od_route_dict = {} 
for od in od_list: 
    od_route_dict[od] = [] 
# print(od_route_dict)  

df_route['route_id'] = [i for i in range(num_route)] 

for route in range(num_route):
    od = df_route.loc[route, 'OD_id'] 
    route = df_route.loc[route, 'route_id'] 
    od_list = od_route_dict[od] 
    if route not in od_list: 
        od_list.append(route) 
        od_route_dict[od] = od_list 

# 14 total OD flow
od_flow_dict = {} 

for od, route_list in od_route_dict.items(): 
    od_flow = 0 
    for route in route_list:  
        od_flow += route_flow[route] 
    # print(od_flow) 
    od_flow_dict[od] = od_flow 

## 模型部分

In [26]:
# 模型部分
### Model Formulation ###
model = gp.Model('path_flow_estimator')

### Variables ###
route_flow_est = model.addVars(num_route, name='route_flow_est') 
route_flow_est_ln = model.addVars(num_route, name='route_flow_est_ln') 

link_flow_est = model.addVars(num_link, name='link_flow_est') 
link_flow_est_two_fold = model.addVars(num_link, name='link_flow_est_two_fold') 

link_tt_est = model.addVars(num_link, name='link_tt') 
link_tt_quad_est = model.addVars(num_link, name='link_tt_quad') 
# route_tt_est = model.addVars(num_route, name='route_tt') 

num_od = len(od_route_dict)
od_flow_var = model.addVars(num_od, name='od_flow_var') 

### Objective function ###
expr = gp.QuadExpr() 

# dispersion parameter
theta = 0.01  # test the model under UE condition （模型参数）
for route in range(num_route): 
    obj_1 = route_flow_est[route] * (route_flow_est_ln[route] - 1) 
    expr +=  1 / theta * obj_1 

for link in range(num_link): 
    tt_tmp = 300 * link_flow_est[link] + 45 * link_flow_est_two_fold[link]  # 根据需要进行改进
    expr += tt_tmp 

model.setObjective(expr, GRB.MINIMIZE) 

# 避免过大方差
# for od, route_list in od_route_dict.items():
#     for route in route_list:
#         expr += (route_flow_est[route] - od_flow_dict[od] / len(route_list)) * (route_flow_est[route] - od_flow_dict[od] / len(route_list))


### Constraints ###
# c0: definitional constraints
model.addConstrs((link_flow_est_two_fold[link] == link_flow_est[link] * link_flow_est[link]
                 for link in range(num_link)))

for route in range(num_route):
    model.addGenConstrLog(route_flow_est[route], route_flow_est_ln[route], name='route_flow_ln %s' % route)

# c1: the observable route flow   ### 无 AVI-sensor 时移除!!! ###   
model.addConstrs((route_flow_est[route] == route_flow[route] 
                 for route in observable_routes), name='route_flow_obs') 

# c2: the estimated link flow 
model.addConstrs((sum([route_flow_est[route] * route_link_indicator[route, link] 
                      for route in range(num_route)]) == link_flow_est[link]
                  for link in range(num_link)), name='link_flow_est')  # all links 

# c3: the observed link flow   ### 无 AVI-sensor 时移除!!! ###
# epslon = 0.05  # allowed measurement error of the link flow 
# model.addConstrs((link_flow_est[link] >= link_flow[link] * (1 - epslon) for link in scanned_links), name='link_flow_obs_l')
# model.addConstrs((link_flow_est[link] <= link_flow[link] * (1 + epslon) for link in scanned_links), name='link_flow_obs_u')

# model.addConstrs((link_flow_est[link] == link_flow[link] for link in scanned_links), name='link_flow_obs')

# c4: od flow (assume it to be known)  
# for od, od_flow in od_flow_dict.items():  
#     route_list = od_route_dict[od] 
#     model.addConstr(sum([route_flow_est[route] for route in route_list]) == od_flow, name='od_flow') 

# 同一个OD 设定不同路径的差异不能太大
for route in range(num_route):
    model.addConstr(route_flow_est[route] <= route_flow[route] * 30) 

### Model Settings ###   
model.setParam(GRB.Param.OutputFlag, 1)  # model.Params.OutputFlag = 0 
model.setParam(GRB.Param.NonConvex, 2)  # model.Params.NonConvex = 2 
model.setParam(GRB.Param.PoolGap, 0.25)   
model.setParam(GRB.Param.TimeLimit, 300)  # model.Params.TimeLimit = 100 

### init values
un_obs_route = [route for route in range(num_route) if route not in observable_routes] 
for route in un_obs_route: 
    route_flow_est[route].start = 1e-3 

### Optimization ### 
model.optimize() 
status = model.Status 

if (status == GRB.OPTIMAL) or (status == GRB.TIME_LIMIT) or (status == GRB.INTERRUPTED): 
    # get the RMSE and the MAPE
    rmse_route_flow, mape_route_flow = 0, 0
    route_flow_result = model.getAttr('x', route_flow_est)
    # print(route_flow_result.values())
    # link_flow_result = model.getAttr('x', link_flow_est)

    for route in range(num_route): 
        # if route in observable_routes:
        #     continue 
        # if (route_flow[route] - 1 >= 1e-3) and ((route_flow_result[route] - 1) < 1e-3):
        #     rmse_route_flow += route_flow[route] ** 2
        #     mape_route_flow += 1
        # else:
        rmse_route_flow += (route_flow[route] - route_flow_result[route]) ** 2 / 100 # RMSE需要缩放回去，MAPE不用
        mape_route_flow += abs(route_flow[route] - route_flow_result[route]) / route_flow[route]

    rmse_route_flow = (rmse_route_flow / num_route) ** 0.5
    mape_route_flow /= num_route
    print('RMSE:', round(rmse_route_flow, 2), 'MAPE: {}%'.format(round(mape_route_flow * 100, 2))) 

#     mape.append(mape_route_flow)
#     rmse.append(rmse_route_flow)

    df_route['route_flow_result'] = route_flow_result.values()  

    # num_sensor = 0  
    # PFE_plot_2(df_route, scenario, num_sensor, observable_routes, rmse_route_flow, mape_route_flow)  
else:  
    print('model has no solution')   

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter PoolGap to 0.25
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 999 rows, 2775 columns and 3861 nonzeros
Model fingerprint: 0xb643a070
Model has 460 quadratic objective terms
Model has 402 quadratic constraints
Model has 460 general constraints
Variable types: 2775 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [4e+01, 3e+02]
  QObjective range [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]

User MIP st